In [1]:
import numpy as np
import pandas as pd
import local_conformal as lc

# Parameters


| | Training|Validation|CDE-Tune|Calibrate| Test |
|-|-|-|-|-|-| 
|Izbicki| 1n |1n|0|1n| 1n |
|Us| .75n|.75n|.75n|.75n| 1n|

In [2]:
n = np.array([100,200,1000, 5000, 10000], dtype = int)
n_sims = 20

In [3]:
def k_values_select(n, n_train):
    out = list(np.arange(2,17, dtype = int)) +\
            [int(n_train/100)] # Izbicki's number
    
    if n > 1000:
        out += list(np.arange(17,33, dtype = int))
        
    return out

In [4]:
def n_gaussians_select(n):
    upper = np.max([20, np.floor(n**(1/3))])
    out = list(np.arange(1, upper, dtype = int))

    return out

In [5]:
def lr_select():
    out = list(10**(-1* np.arrange(2,7)))
    return out

In [6]:
def hidden_select(n):
    n_based_value = np.floor(n**.5)
    if (n_based_value > 10):
        upper = int(np.min([50, n_based_value]))
    else:
        upper = 10
    out = list(np.arange(5, upper, dtype = int))

    return out

In [7]:
def quantiles_vs_expected():
    """
    for each observations cs, look at their groups in calibration mat cs
    and calculate mean(cs_obs >= cs_cal)
    
    Todo: this could potentially be fit into `thresholds_per_group`?
    
    """
    pass

In [9]:

#     def full_process(n, n_sims, sigma  = 4):
#         # get ranges of all parameters

#         # or write a python script, and run this in parallel (slurm style)?

#         all_info_across = pd.DataFrame(columns = ["n_sim",
#                                                "approach_type",
#                                                "mdn_n_gaussian",
#                                                "mdn_lr",
#                                                "mdn_hidden1",
#                                                "mdn_hidden2",
#                                                "mdn_train_error",
#                                                "mdn_val_error",
#                                                "q_lr",
#                                                "q_hidden1",
#                                                "q_hidden2",
#                                                "q_train_error",
#                                                "q_val_error",
#                                                "k",
#                                                "x_group",
#                                                "validity_truth",
#                                                "validity_test",
#                                                "efficiency_truth"
#                                                ])
#         for sim_idx in np.arange(n_sims):
#             data_all = lc.data_generation(n, sigma = sigma)
#             inner_info_iz = inner_process_izbicki(data_all, ...parameters) 
#             inner_info_us = inner_process_us(data_all, ...parameters) 

#             inner_info["n_sim"] = s_idx
#             inner_info2 = inner_info[["n_sim",
#                                       "approach_type",
#                                        "mdn_n_gaussian",
#                                        "mdn_lr",
#                                        "mdn_hidden1",
#                                        "mdn_hidden2",
#                                        "mdn_train_error",
#                                        "mdn_val_error",
#                                        "q_lr",
#                                        "q_hidden1",
#                                        "q_hidden2",
#                                        "q_train_error",
#                                        "q_val_error",
#                                        "k",
#                                        "x_group",
#                                        "validity_truth",
#                                        "validity_test",
#                                        "efficiency_truth"
#                                        ]]

#             all_info_across = all_info_across.append(inner_info2)

#         return all_info_across 

In [10]:
n = 200
data_all = lc.data_generation(n, 4)

data_izbicki_list = lc.stratified_data_splitting(data_all,
                                                prop_vec = np.array([1,1,0,1,1])/4)
x_train_iz,y_train_iz, g_train_iz = lc.torchify_data(data_izbicki_list[0].x, 
                                        data_izbicki_list[0].y, 
                                        data_izbicki_list[0].group_info)
x_val_iz,y_val_iz, g_val_iz = lc.torchify_data(data_izbicki_list[1].x, 
                                        data_izbicki_list[1].y, 
                                        data_izbicki_list[1].group_info)
x_cal_iz,y_cal_iz, g_val_iz = lc.torchify_data(data_izbicki_list[3].x, 
                                        data_izbicki_list[3].y, 
                                        data_izbicki_list[3].group_info)
x_test_iz,y_test_iz, g_test_iz = lc.torchify_data(data_izbicki_list[4].x, 
                                        data_izbicki_list[4].y, 
                                        data_izbicki_list[4].group_info)



column_order = [
               "approach_type",
               "mdn_n_gaussian",
               "mdn_lr",
               "mdn_hidden1",
               "mdn_hidden2",
               "mdn_train_error",
               "mdn_val_error",
               "q_lr",
               "q_hidden1",
               "q_hidden2",
               "q_train_error",
               "q_val_error",
               "k",
               "x_group",
               "validity_truth",
               "validity_test",
               "efficiency_truth"
               ]
all_info_out = pd.DataFrame(columns = column_order)

all_info_out


n_guassian, lr_1, hidden1_1, hidden2_1 = 1, 1e-03, 10, 10
epochs = 10000

In [11]:
mdn_model_iz, mdn_opt_iz, val_error_iz = lc.tune_first_nn(
                                                 x_train = x_train_iz,
                                                 y_train = y_train_iz,
                                                 x_val = x_val_iz,
                                                 y_val = y_val_iz,
                                                 epochs = epochs,
                                                 n_gaussians=n_guassian,
                                                 n_hidden1 = hidden1_1,
                                                 n_hidden2 = hidden2_1,
                                                 lr = lr_1, verbose = True)

|#####################################| (Time:  0:00:19, Elapsed Time: 0:00:19)


In [12]:
y_range = np.array([-25,25]).reshape((-1,1))
y_splits = 10000
t_range_size = 500

In [13]:
mdn_model_iz.prep_for_cde(y_range, y_splits)
model_y_range_iz = (mdn_model_iz.y_range[0],
                mdn_model_iz.y_range[1])
model_n_grid_iz = mdn_model_iz.n_grid
y_grid_iz = np.linspace(model_y_range_iz[0], model_y_range_iz[1],
                       model_n_grid_iz)

In [14]:
y_delta_iz = (mdn_model_iz.y_range[1] - mdn_model_iz.y_range[0]) /\
            mdn_model_iz.n_grid

cde_train_iz = mdn_model_iz.cde_predict_grid(x_train_iz).detach().numpy() 
cde_cal_iz = mdn_model_iz.cde_predict_grid(x_cal_iz).detach().numpy()
cde_test_iz = mdn_model_iz.cde_predict_grid(x_test_iz).detach().numpy()

t_grid = np.linspace(cde_train_iz.min(), cde_train_iz.max(), t_range_size)


profile_density_train_iz = lc.profile_density(cde_train_iz, 
                                           t_grid = t_grid,
                                           z_delta = y_delta_iz)

profile_density_cal_iz = lc.profile_density(cde_cal_iz,
                                         t_grid = t_grid,
                                         z_delta = y_delta_iz)

profile_density_test_iz = lc.profile_density(cde_test_iz,
                                         t_grid = t_grid,
                                         z_delta = y_delta_iz)

In [15]:
k = 3
quantiles = np.arange(1,500)/500

In [16]:
kmeans_model_iz, _, _ = lc.profile_grouping(profile_density_train_iz, 
                                            k = k)

_, _, grouping_cal_iz = lc.profile_grouping(profile_test = profile_density_cal_iz, Kmeans_model = kmeans_model_iz)
_, _, grouping_test_iz= lc.profile_grouping(profile_test = profile_density_test_iz, Kmeans_model = kmeans_model_iz)



In [17]:

# get distribution of conformal scores -------
cs_cal_iz = mdn_model_iz.cde_predict(x_cal_iz, y_cal_iz)

df_cs_group_cal_iz = pd.DataFrame(data = {"x": x_cal_iz.numpy().ravel(),
                                            "cs": cs_cal_iz.detach().numpy(),
                                            "grouping":grouping_cal_iz})


# will need to calculate cuts per each threshold per group

cs_test_iz = mdn_model_iz.cde_predict(x_test_iz, y_test_iz)


df_cs_group_test_iz = pd.DataFrame(data = {"x": x_test_iz.numpy().ravel(),
                                           "cs": cs_test_iz.detach().numpy(),
                                           "grouping":grouping_test_iz})

thresholds_mat_test_iz, _ = lc.thresholds_per_group(df_cs_group_cal_iz,
                                              desired_props = quantiles,
                                              append = df_cs_group_test_iz)



In [18]:
true_cde_test_iz = lc.true_cde_out(x_test_iz.numpy().ravel(), y_grid_iz)

In [19]:
true_thresholds_test_iz = lc.true_thresholds_out(true_cde = true_cde_test_iz, 
                                                z_delta = y_delta_iz,
                                                expected_prop = quantiles)

In [20]:
def difference_validity_and_efficiency(true_cde, predict_grid,
                                       true_grid,
                                       thresholds_predict,
                                       thresholds_true,
                                       expected_prop,
                                       z_delta = 1,
                                       verbose = True):
    """
    Calculates difference in validity and efficiency between predicted and true
    estimates at an individual level.

    Arguments:
    ----------
    true_cde : numpy array (n, p). CDE estimates across a range of z values
        that are separated by a common `z_delta`. Each row relates to a single
        observation of a CDE function, and the columns of the array should
        correspond to the CDE_i(y_j) evaluation.
    predict_grid : numpy array (n, p). Predicted scores (e.g. HDP, CDE values,
        etc.) across the same range of z values as `true_cde` with same
        row/column structure as well
    true_grid: numpy array (n, p). Same as `predict_grid` but the true scores
        under the oracle (aka knowing `true_cde`)
    thresholds_predict : numpy vector (n, m). Threshold cutoffs associated with the
        predict_grid scores for each row of the above arrays (`true_cde`,
        `predict_grid`, `true_grid`). The columns are associated with difference
        cutoffs.
    thresholds_true : numpy vector (n, m). Threshold cutoffs associated with the
        true_grid scores for each row of the above arrays (`true_cde`,
        `predict_grid`, `true_grid`). The columns are associated with difference
        cutoffs.
    expected_prop : numpy vector (m,). Amount of mass expected to be contained
        in each level set defined by columns of `thresholds`
    z_delta : difference between the range of z values used to create above
        arrays (`true_cde`, `predict_grid`, `true_grid`)
    verbose : boolean, logic if we should report progress of analysis

    Returns:
    --------
    a tuple with the following info:
        validity_error : numpy array (n, m). For each row (i) of the above numpy
            matrices, we look at the absolute error between the expected_prop
            verse actual mass in the level set. The columns are ranging across
            the different thresholds and expected_prop values.
        efficiency_error : numpy array (n, m). For each row (i) of the above
            numpy matrices, we look at the lebegue measure of the set difference
            between predicted level set and the true level set.

    Details:
    --------
    This function can do both HDP and CDE style cutoffs (can actually do any
    conformal score function evaluated on the same grid).

    For **validity**: we calculate the discrete probability mass of the
    predicted level set (at each given threshold) related to the expected_prop.

    For **efficiency**: we calculate the lebegue set difference beween the
    predicted level set vs the true level set (as defined with `true_grid`)
    """

    # dimension checks -------

    assert true_cde.shape == predict_grid.shape and \
        true_cde.shape == true_grid.shape, \
        "dimensions of true_cde, predict_grid and true_grid should be the same"

    assert thresholds_predict.shape[1] == expected_prop.shape[0] and \
        thresholds_predict.shape == thresholds_true.shape, \
        "number of columns of thresholds_{predict, true} should be the same "+\
        "as the length of expected_prop"

    assert thresholds_predict.shape[0] == true_cde.shape[0], \
        "thresholds_{predict, true} should have the same number of rows as "+\
        "true_cde"

    # verbosity design (across thresholds) ---------
    if verbose:
        bar = progressbar.ProgressBar(widgets = [ progressbar.Bar(),
                                              ' (', progressbar.ETA(), ", ",
                                              progressbar.Timer(), ')'])
        threshold_iter = bar(np.arange(thresholds_predict.shape[1]))
    else:
        threshold_iter = range(thresholds_predict.shape[1])


    validity_error = -1 * np.ones((true_cde.shape[0],
                                   thresholds_predict.shape[1]))
    efficiency_error = -1 * np.ones((true_cde.shape[0],
                                     thresholds_predict.shape[1]))

    for t_idx in threshold_iter:
        threshold_predict_vec = thresholds_predict[:,t_idx]

        predict_indicator_ge_threshold = predict_grid >= \
            np.repeat(threshold_predict_vec.reshape((-1,1)),
                      predict_grid.shape[1], axis = 1)


        # oracle validity: with the underlying distribution ---------
        expected_p = expected_prop[t_idx]

        mask_cde_value = true_cde * predict_indicator_ge_threshold
        rowwise_mass = mask_cde_value.sum(axis = 1) * z_delta

        validity_error[:, t_idx] = np.abs(rowwise_mass - expected_p)


        # efficiency: compared to truth optimal set ---------
        threshold_true_vec = thresholds_true[:,t_idx]


        true_indicator_ge_threshold = true_grid >= \
            np.repeat(threshold_true_vec.reshape((-1,1)),
                      predict_grid.shape[1], axis = 1)

        level_set_diff = true_indicator_ge_threshold == predict_indicator_ge_threshold
        rowwise_diff = level_set_diff.sum(axis = 1)
        efficiency_error[:, t_idx] = rowwise_diff * z_delta


    return validity_error, efficiency_error

In [21]:
v_mat_iz, e_mat_iz = lc.difference_validity_and_efficiency(true_cde = true_cde_test_iz,
                                                 predict_grid = cde_test_iz,
                                                 true_grid = true_cde_test_iz,
                                                 thresholds_predict = thresholds_mat_test_iz,
                                                 thresholds_true = true_tresholds_test_iz,
                                                 expected_prop = quantiles, 
                                                 z_delta = y_delta_iz,
                                                 verbose = False)

In [23]:
test_bin_raw_iz = lc.difference_actual_validity(df_cs_group_test_iz,
                                           thresholds_mat_test_iz)

In [ ]:
def 

In [ ]:


def inner_process_izbicki(data_all,
                         k_range, n_gaussians_range, lr_range, hidden1_range, 
                          hidden2_range, verbose = T, 
                          epochs = 10000, 
                          y_range = np.array([-25,25]).reshape((-1,1)),
                          y_splits = 10000,
                          quantiles = np.arange(1,500)/500,
                          t_range_size = 500):
    
    data_izbicki_list = lc.stratified_data_splitting(data_all,
                                                    prop_vec = np.array([1,1,0,1,1])/4)
    x_train_iz,y_train_iz, g_train_iz = lc.torchify_data(data_izbicki_list[0].x, 
                                            data_izbicki_list[0].y, 
                                            data_izbicki_list[0].group_info)
    x_val_iz,y_val_iz, g_val_iz = lc.torchify_data(data_izbicki_list[1].x, 
                                            data_izbicki_list[1].y, 
                                            data_izbicki_list[1].group_info)
    x_cal_iz,y_cal_iz, g_val_iz = lc.torchify_data(data_izbicki_list[3].x, 
                                            data_izbicki_list[3].y, 
                                            data_izbicki_list[3].group_info)
    x_test_iz,y_test_iz, g_test_iz = lc.torchify_data(data_izbicki_list[4].x, 
                                            data_izbicki_list[4].y, 
                                            data_izbicki_list[4].group_info)

    
    column_order = [
                   "approach_type",
                   "mdn_n_gaussian",
                   "mdn_lr",
                   "mdn_hidden1",
                   "mdn_hidden2",
                   "mdn_train_error",
                   "mdn_val_error",
                   "q_lr",
                   "q_hidden1",
                   "q_hidden2",
                   "q_train_error",
                   "q_val_error",
                   "k",
                   "x_group",
                   "validity_truth",
                   "validity_test",
                   "efficiency_truth"
                   ]
    all_info_out = pd.DataFrame(columns = column_order)
    
    
    for n_guassian, lr_1, hidden1_1, hidden2_1 in
        itertools.product(n_gaussians_range, lr_range, hidden1_range, hidden2_range): # no k_range and only MDN parameters currently			

        # izbicki --------
        mdn_model_iz, mdn_opt_iz, train_error_iz, val_error_iz = \
                                        lc.tune_first_nn(x_train = x_train_iz,
                                                         y_train = y_train_iz,
                                                         x_val = x_val_iz,
                                                         y_val = v_val_iz,
                                                         epochs = epochs,
                                                         n_guassians=n_guassian,
                                                         n_hidden1 = hidden1_1,
                                                         n_hidden2 = hidden2_1,
                                                         lr = lr_1)
        mdn_model_iz.prep_for_cde(y_range, y_splits)
        model_y_range_iz = (mdn_model_iz.y_range[0],
                         mdn_model_iz.y_range[1])
        model_n_grid_iz = mdn_model_iz.n_grid
        y_grid_iz = np.linspace(model_y_range_iz[0], model_y_range_iz[1],
                                model_n_grid_iz)

        # izbicki clustering -----
        y_delta_iz = (mdn_model_iz.y_range[1] - mdn_model_iz.y_range[0]) /\
                    mdn_model_iz.n_grid

        cde_train_iz = mdn_model_iz.cde_predict_grid(x_train_iz).detach().numpy() 
        cde_cal_iz = mdn_model_iz.cde_predict_grid(x_cal_iz).detach().numpy()
        cde_test_iz = mdn_model_iz.cde_predict_grid(x_test_iz).detach().numpy()
        
        t_grid = np.linspace(cde_mat.min(), cde_mat.max(), t_range_size)


        profile_density_train_iz = lc.profile_density(cde_train_iz, 
                                                   t_grid = t_grid,
                                                   z_delta = y_delta)

        profile_density_cal_iz = lc.profile_density(cde_cal_iz,
                                                 t_grid = t_grid,
                                                 z_delta = y_delta)

        profile_density_test_iz = lc.profile_density(cde_test_iz,
                                                 t_grid = t_grid,
                                                 z_delta = y_delta)
        for k_val in k_range:

            kmeans_model_iz, _, _ = lc.profile_grouping(profile_density_train_iz, 
                                                        k = k)
            _, _, grouping_cal_iz = lc.profile_grouping(profile_test = profile_density_cal_iz, 
                                                        Kmeans_model = kmeans_model_iz)
            _, _, grouping_test_iz = lc.profile_grouping(profile_test = profile_density_test_iz, 
                                                        Kmeans_model = kmeans_model_iz)



            # get distribution of conformal scores -------
            cs_cal_iz = mdn_model_iz.cde_predict(x_cal_iz, y_cal_iz)

            df_cs_group_cal_iz = pd.DataFrame(data = {"x": x_cal_iz.numpy().ravel(),
                                                      "cs": cs_cal_iz.detach().numpy(),
                                                      "grouping":grouping_cal_iz})


            # will need to calculate cuts per each threshold per group
            cs_test_iz = mdn_model_iz.cde_predict(x_test_iz, y_test_iz)
            
            df_cs_group_test_iz = pd.DataFrame(data = {"x": x_test_iz.numpy().ravel(),
                                                       "cs": cs_test_iz.detach().numpy(),
                                                       "grouping":grouping_test_iz})

            thresholds_mat_test_iz, _ = lc.thresholds_per_group(df_cs_group_cal_iz,
                                                          desired_props = quantiles,
                                                          append = df_cs_group_test_iz)


            true_cde_test_iz = true_cde_out(x_test_iz.numpy().ravel(), y_grid_iz)
            
            true_thresholds_test_iz = lc.true_thresholds_out(true_cde = true_cde_test_iz, 
                                                z_delta = y_delta_iz,
                                                expected_prop = quantiles)

            v_mat_iz, e_mat_iz = lc.difference_validity_and_efficiency(true_cde = true_cde_test_iz,
                                                             predict_grid = cde_test_iz,
                                                             true_grid = true_cde_test_iz,
                                                             thresholds_predict = thresholds_mat_test_iz,
                                                             thresholds_true = true_tresholds_test_iz,
                                                             expected_prop = quantiles, 
                                                             z_delta = y_delta_iz,
                                                             verbose = False)
            # write a function to evalulate test data proportions discretely
            
            v_test_bin_raw_iz = lc.difference_actual_validity(df_cs_group_test_iz,
                                                              thresholds_mat_test_iz)
            #^ this might be able to be put in diff_v_and_e...
            # COME HERE!
            #^ this will need to be transformed differently than v_mat_iz and e_mat_iz...
            #^ we'll still need to process relative to desired outcome...
            
            ## g_test_iz (should this function actually be called in difference_.. functions? (yes))
            _, v_average_iz = lc.average_within_groups(g_test_iz, v_mat_iz)
            _, e_average_iz = lc.average_within_groups(g_test_iz, e_mat_iz)
            
            
            
            #^still need to be averaged afterwards...
            # average across true groups
            # also do we melt that information into a df to be stored? (yes)
            # ^ see average_within_groups for pd.concat approach... (hopefully)
            

            # how to combine items across ... 
            inner_info = pd.DataFrame({"mdn_n_gaussian": "iz",
                                       "mdn_lr": lr_1,
                                       "mdn_hidden1": hidden1_1,
                                       "mdn_hidden2": hidden2_1,
                                       "mdn_train_error": train_error_iz,
                                       "mdn_val_error": val_error_iz,
                                       "q_lr": None,
                                       "q_hidden1": None,
                                       "q_hidden2": None,
                                       "q_train_error": None,
                                       "q_val_error": None,
                                       "k": k_val,
                                       "x_group": ...vec,
                                       "validity_truth": ...vec,
                                       "validity_test": ...vec,
                                       "efficiency_truth": ...vec},
                                      columns = column_order)
            
        






In [ ]:


    
def inner_process_us(data_all, k_range, n_gaussians_range, lr_range, 
                     hidden1_range, hidden2_range, verbose = T, 
                     epochs = 10000, 
                     y_range = np.array([-25,25]).reshape((-1,1)),
                     y_splits = 10000,
                     quantiles = np.arange(1,500)/500, #we are using these quantiles relative to conformal cuts... that isn't what was expected
                     t_range_size = 500):

    
    data_us_list = lc.stratified_data_splitting(data_all, 
                                                prop_vec = np.array([.75,.75,.75,.75,1])/4)
    
    x_train_us,y_train_us, g_train_us = lc.torchify_data(data_us_list[0].x, 
                                        data_us_list[0].y, 
                                        data_us_list[0].group_info)
    x_val_us,y_val_us, g_val_us = lc.torchify_data(data_us_list[1].x, 
                                        data_us_list[1].y, 
                                        data_us_list[1].group_info)
    x_cde_cal_us, y_cde_cal_us, lc.torchify_data(g_cde_cal_us = data_us_list[2].x, 
                                        data_us_list[2].y, 
                                        data_us_list[2].group_info)
    x_cal_us,y_cal_us, g_cal_us = lc.torchify_data(data_us_list[3].x, 
                                        data_us_list[3].y, 
                                        data_us_list[3].group_info)
    x_test_us,y_test_us, g_test_us = lc.torchify_data(data_us_list[4].x, 
                                        data_us_list[4].y, 
                                        data_us_list[4].group_info)
    
    column_order = [
                   "approach_type",
                   "mdn_n_gaussian",
                   "mdn_lr",
                   "mdn_hidden1",
                   "mdn_hidden2",
                   "mdn_train_error",
                   "mdn_val_error",
                   "q_lr",
                   "q_hidden1",
                   "q_hidden2",
                   "q_train_error",
                   "q_val_error",
                   "k",
                   "x_group",
                   "validity_truth",
                   "validity_test",
                   "efficiency_truth"
                   ]
    
    all_info_out = pd.DataFrame(columns = column_order)
    

        
    
    
    for n_guassian, lr_1, hidden1_1, hidden2_1 in
        itertools.product(n_gaussians_range, lr_range, 
                          hidden1_range, hidden2_range): # no k_range and only MDN parameters currently

        # build MDN model (record training and validation loss) ------
        
        # us ---------
        mdn_model_us, mdn_opt_us, train_error_us, val_error_us = lc.tune_first_nn(
                                                         x_train = x_train_us,
                                                         y_train = y_train_us,
                                                         x_val = x_val_us,
                                                         y_val = v_val_us,
                                                         epochs = epochs,
                                                         n_guassians=n_guassian,
                                                         n_hidden1 = hidden1_1,
                                                         n_hidden2 = hidden2_1,
                                                         lr = lr_1)
        mdn_model_us.prep_for_cde(y_range, y_splits)
        model_y_range_us = (mdn_model_us.y_range[0],
                         mdn_model_us.y_range[1])
        model_n_grid_us = mdn_model_us.n_grid
        y_grid_us = np.linspace(model_y_range_us[0], model_y_range_us[1], model_n_grid_us)
        
        # prepping for second model:
        cde_train_us = mdn_model_us.cde_predict_grid(x_train_us).detach().numpy()
        cde_val_us = mdn_model_us.cde_predict_grid(x_val_us).detach().numpy()
        
        hpd_train_us = lc.hpd_coverage(cde_train_us, y_grid_us, y_train_us.numpy())
        hpd_val_us = lc.hpd_coverage(cde_val_us, y_grid_us, y_val_us.numpy())
        
        
        # TODO: tune_first_nn & tune_second_nn: record training error too!
        
        for lr_2, hidden1_1, hidden2_1 in itertools.product(lr_range, hidden1_range, hidden2_range): 
            # build Quantile Reg model (record training and validation loss) ---
            
            q_model, q_opt, q_train_error_us, q_val_error_us = lc.tune_second_nn(x_train = x_train_us,
                                                            y_train = hpd_train_us,
                                                            x_val = x_val_us,
                                                            y_val = hpd_val_us,
                                                            epochs = epochs,
                                                            quantiles = quantiles,
                                                            n_hidden1 = hidden1_2,
                                                            n_hidden2 = hidden2_2,
                                                            lr = lr_2,
                                                            verbose = False)
            
            hpd_estimate_cal_us = q_model(x_cal_us).detach().numpy()
            hpd_estimate_test_us = q_model(x_test_us).detach().numpy()
            
            # apply groupings  -----
            for k_val in k_range:
                kmeans_model_us, grouping_cal_us, _ = \
                    lc.profile_grouping(hpd_estimate_cal_us, k = k)

                _, grouping_test_iz, _ = lc.profile_grouping(profile_density_test_iz, ...kmeans_model_us)
            
                
                # need to convert CDE to HDP for our approach...
                
                df_cs_group_cal_us = pd.DataFrame(data = {"x": x_cal_us.numpy().ravel(),
                                                        "cs": ...,
                                                        "grouping":grouping_cal_us})
            


                df_cs_group_test_us = pd.DataFrame(data = {"x": x_test_us.numpy().ravel(),
                                                           "cs": ...,
                                                           "grouping":grouping_test_us})
            
                thresholds_mat_test_us, _ = thresholds_per_group(df_cs_group_cal_us,
                                                              desired_props = quantiles,
                                                              append = df_cs_group_test_us)

                v_mat_us, e_mat_us = diference_validity_and_efficiency(true_cde = ...true_cde_test_us,
                                                     predict_grid = ...hpd_grid_test_us,
                                                     true_grid = ...true_hpd_test_us,
                                                     thresholds_predict = ...thresholds_hpd_test_us,
                                                     thresholds_grid = ...true_hdp_test_us,
                                                     expected_prop = quantiles, 
                                                     z_delta = y_delta_us,
                                                     verbose = False)
            
                inner_info = pd.DataFrame({"mdn_n_gaussian": "us",
                                       "mdn_lr": lr_1,
                                       "mdn_hidden1": hidden1_1,
                                       "mdn_hidden2": hidden2_1,
                                       "mdn_train_error": train_error_us,
                                       "mdn_val_error": val_error_us,
                                       "q_lr": lr_2,
                                       "q_hidden1": hidden1_2,
                                       "q_hidden2": hidden2_1,
                                       "q_train_error": q_train_error_us,
                                       "q_val_error": q_val_error_us,
                                       "k": k_val,
                                       "x_group": ...vec,
                                       "validity_truth": ...vec,
                                       "validity_test": ...vec,
                                       "efficiency_truth": ...vec},
                                          columns = column_order)
                
        